In [ ]:
# how to stream msgs in streamlit

In [1]:
import pandas as  pd
from langchain_groq import ChatGroq
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonREPLTool
from dotenv import load_dotenv

In [5]:
load_dotenv()
df = pd.read_csv(r"E:\Machine learning\Sources\telecom data.csv")
model = ChatGroq(model="llama3-70b-8192")

In [3]:
SYSTEM_PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a python data visualizer which uses python repl tool to generate plotly visualization with title and labelled axis.
Generate the plot : {kind} for the features : {features} only.

NOTE:
1. Make the plot only with pandas,numpy,plotly.express,streamlit libraries.
2. The generated plot should be clear and colorful with title and labeled axes.
3. Execute only 1 plot in streamlit using st.plotly_chart function
4. Don't generate multiple charts.

Also, after execution of plot, interpret the graph with reference to the dataset and show it using streamlit write function.<|eot_id|>
"""

prompt_template = ChatPromptTemplate.from_template(template=SYSTEM_PROMPT)

In [6]:
python_repl_tool = PythonREPLTool(name="Python REPL",
    description="""A python shell which execute code for data visualization using the following libraries i.e. 
    pandas, numpy, streamlit and plotly.express only. Execute streamlit code in shell to display the graph using plotly_chart function.""".strip())

model_with_tools = model.bind_tools(tools=[python_repl_tool])


In [29]:
def run_dataviz_agent(kind,features,dataframe,prompt=SYSTEM_PROMPT):
    python_repl_tool = PythonREPLTool(name="Python REPL",
    description="""A python shell which execute code for data visualization using the following libraries i.e. 
    pandas, numpy, streamlit and plotly.express only. Execute streamlit code in shell to display the graph using plotly_chart function.""".strip())

    tool_agent = create_pandas_dataframe_agent(
    llm=model.bind_tools(tools=[python_repl_tool]),
    df=dataframe,
    extra_tools=[python_repl_tool],
    agent_type="tool-calling",
    allow_dangerous_code=True,
    verbose=True)

    input_prompt = prompt.format(kind=kind,
                                 features=features)
    
    tool_agent.invoke(input_prompt)
    return

In [30]:
run_dataviz_agent(kind="scatter plot",features="tenure,income",dataframe=df)



> Entering new AgentExecutor chain...

Invoking: `Python REPL` with `{'query': "import pandas as pd\nimport numpy as np\nimport plotly.express as px\n\nfig = px.scatter(df, x='tenure', y='income')\nfig.update_layout(title='Scatter plot of tenure vs income', xaxis_title='Tenure', yaxis_title='Income')\nst.plotly_chart(fig)"}`


NameError: name 'df' is not defined

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>{"tool_calls":[{"id":"pending","type":"function","function":{"name":"python_repl_ast"},"parameters":{"query":"df = df"}}},{"id":"call_9t5v","type":"function","function":{"name":"Python REPL"},"parameters":{"query":"import pandas as pd\\nimport numpy as np\\nimport plotly.express as px\\nfig = px.scatter(df, x=\'tenure\', y=\'income\')\\nfig.update_layout(title=\'Scatter plot of tenure vs income\', xaxis_title=\'Tenure\', yaxis_title=\'Income\')\\nst.plotly_chart(fig)"}}]}</tool-use>'}}

In [18]:
df.head()

,region,tenure,age,marital,address,income,ed,employ,retire,gender,reside,custcat
0,2,13,44,1,9,64.0,4,5,0.0,0,2,1
1,3,11,33,1,7,136.0,5,5,0.0,0,6,4
2,3,68,52,1,24,116.0,1,29,0.0,1,2,3
3,2,33,33,0,12,33.0,2,0,0.0,1,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,4,3
